In [ ]:
# windows only hack for graphviz path 
import os
for path in os.environ['PATH'].split(os.pathsep):
    if path.endswith("Library\\bin"):
        os.environ['PATH']+=os.pathsep+os.path.join(path, 'graphviz')

In [ ]:
import tensorflow as tf
import numpy as np

# 下面兩個是用來輔助圖形化
from IPython.display import display
from tfdot import tfdot

## 常數及節點

In [ ]:
tf.constant(42)

In [ ]:
tf.constant(42.)

In [ ]:
tf.constant([42])

In [ ]:
matrix1 = tf.constant([[3., 3.]])

matrix2 = tf.constant([[2.],[2.]])

matrix1, matrix2

In [ ]:
product = tf.matmul(matrix1, matrix2)
product

In [ ]:
tfdot()

這些東西的單位叫做 graph

In [ ]:
graph = tf.get_default_graph()
graph

In [ ]:
product.graph

In [ ]:
# 從 graph 得到 tensor
graph.get_tensor_by_name('MatMul:0')

## Q:
試試看其他名稱

### Operator

In [ ]:
graph.get_operations()

In [ ]:
product.op

In [ ]:
# 運算的輸出節點
product.op.outputs

In [ ]:
# 運算的輸入節點
list(product.op.inputs)

## Q
* 試試看將 numpy ndarray 轉成 tf.constant
* 建立一個 matrix1 和 matrix2 逐項相乘的節點(猜一下是 tf.什麼)
* 用 `tf.reset_default_graph()` 清掉 default graph 看看，會發生什麼事情？
* 再跑一下 tfdot 看看

In [ ]:
# 建立逐項相乘的參考方式
# %load q_constant_mul.py

## Session

如果圖是靜態的描述操作，動態的狀態就是 Session

In [ ]:
# 建立一個 Session
sess = tf.Session()
# 在這個 Session 中執行 product 並且印出結果。
print(sess.run(product))
# 結束這個 Session
sess.close()

### 也可以用 context manager 的寫法

In [ ]:
with tf.Session() as sess:
    print(sess.run(product))
    # 也可以用
    print(product.eval())

## Q
* 用 numpy 檢查結果
* 把前面所有的 operation 都跑一遍

In [ ]:
# 計算所有結果
%run -i q_run_all_op.py

### Device context 
可以設定 device context

In [ ]:
with tf.Session() as sess:
    with tf.device("/cpu:0"):
        print(sess.run(product))

## Q
清掉前面的 default graph， 然後用不同的方式來計算 $1+2+\cdots+10$

* 用公式 $10\cdot(10+1)/2$
* 用 `tf.reduce_sum`
* 用 `tf.add_n`
* 用 `tf.matmul`
* 用 python 迴圈建立 graph


In [ ]:
# 參考答案
%run -i q_sum.py

### Interactive session
在 notebook 中，可以用 interactive session， 就不用特別指名 session 了。 比較方便。

In [ ]:
# 重新設定環境
tf.reset_default_graph()
# 設定 default session
sess =  tf.InteractiveSession()

常數太無聊， 試試看可以改變輸入的運算

In [ ]:
# place holder, 先佔位子
a = tf.placeholder(tf.float32, name="this_is_a")
b = tf.placeholder(tf.float32, name="this_is_b")
s = tf.add(a, b)
display(tfdot())
s

直接執行
```python
s.eval()
```
會爆掉，因為佔了位子沒人來

所以要放東西進去

In [ ]:
s.eval({a:2, b: 5})

或者

In [ ]:
sess.run(s, {a:[1,2], b:[3,4]})

In [ ]:
sess.close()

## Variable
傳遞資訊不是免費的

變數：存東西在 session 的空間


In [ ]:
# 重新設定 graph 環境和 default session
tf.reset_default_graph()
sess = tf.InteractiveSession()
# 計數器
state = tf.Variable(0, name="state")

# 新的節點 計數器+1
new_value =  tf.add(state, tf.constant(1, name='one'), name='new_value')
# 更新 state
update = tf.assign(state, new_value)
# 變數初始化，這也是一個節點
init_op = tf.global_variables_initializer()
tfdot()

上面都是靜態的，下面才開始在 session 中執行

In [ ]:
init_op.run()
# or sess.run(init_op)
print(state.eval())

In [ ]:
for _ in range(300):
    #執行更新
    print(update.eval())

In [ ]:
state.eval()

In [ ]:
sess.run([update]*10)

In [ ]:
sess.close()

### Initialize from another variable

In [ ]:
# 重設環境
tf.reset_default_graph()
sess = tf.InteractiveSession()

# 第一個變數 weights
weights = tf.Variable(tf.random_normal((10,), stddev=0.35), name='weights')
# 想要讓 w2 的初始值設定成和 weights 一樣 
w1 = tf.Variable(weights.initialized_value(), name ='w1')
# 想將 w_twice 設定為 weights 的兩倍
w2 = tf.Variable(weights.initialized_value()*tf.constant(2., name='two'), name="w2")
tfdot()

In [ ]:
init_op = tf.global_variables_initializer()
init_op.run()

for v in tf.global_variables():
    print(v.name, v)
    print(v.eval())
    

In [ ]:
sess.close()

### 流程控制
https://www.tensorflow.org/api_guides/python/control_flow_ops

## Q
試著用 `tf.while_loop` 來計算 $1+2+\cdots+10$
(如何將結果放入一個 `tf.Variable` 中)

## Higher Order Function

https://www.tensorflow.org/api_guides/python/functional_ops

## Q
計算 $1+2+\cdots+10$

## Eager Executation

https://www.tensorflow.org/programmers_guide/eager
## Q

* 計算 $1+2+\cdots+10$
* Project Euler 第一題 https://projecteuler.net/problem=1